<a href="https://colab.research.google.com/github/ag4267research1/Solving-Poisson-s-Equation-with-GNO/blob/main/GNOPDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neuraloperator --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 72.7 MB/s eta 0:00:00


In [ ]:
import os, pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import inspect
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Correct imports:
from neuralop.layers.gno_block import GNOBlock
from neuralop.data.transforms.normalizers import UnitGaussianNormalizer
from neuralop.training import Trainer



In [ ]:
# Folder in the Colab VM (not Drive)
!mkdir -p /content/nonlinear_poisson
!cd /content/nonlinear_poisson

# Download with progress bar (percentage + speed)
!wget --show-progress \
  "https://zenodo.org/records/15001788/files/nonlinear_poisson.obj?download=1" \
  -O nonlinear_poisson.obj




--2025-12-07 18:59:03--  https://zenodo.org/records/15001788/files/nonlinear_poisson.obj?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.75, 137.138.52.235, 188.185.43.153, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9608410281 (8.9G) [application/octet-stream]
Saving to: ‘nonlinear_poisson.obj’

nonlinear_poisson.o 100%[===================>]   8.95G  19.1MB/s    in 8m 2s   

2025-12-07 19:07:05 (19.0 MB/s) - ‘nonlinear_poisson.obj’ saved [9608410281/9608410281]



In [142]:
# Show file size
!ls -lh nonlinear_poisson.obj

-rw-r--r-- 1 root root 9.0G Dec  7 19:07 nonlinear_poisson.obj


In [143]:
DATA_PATH = "/content/nonlinear_poisson.obj"
print("Exists?", os.path.exists(DATA_PATH))

Exists? True


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

DATA_PATH = "/content/nonlinear_poisson.obj"
print("Exists?", os.path.exists(DATA_PATH))

with open(DATA_PATH, "rb") as f:
    raw_data = pickle.load(f)

print("Type:", type(raw_data))
print("Number of PDE samples:", len(raw_data))

sample = raw_data[0]
print("Sample keys:", sample.keys())

Device: cuda
Exists? True
Type: <class 'list'>
Number of PDE samples: 10000
Sample keys: dict_keys(['train_points_boundary', 'train_values_boundary', 'train_source_terms_boundary', 'train_bc_boundary', 'train_points_domain', 'train_values_domain', 'train_distances_domain', 'train_source_terms_domain', 'train_bc_domain', 'val_points_boundary', 'val_values_boundary', 'val_source_terms_boundary', 'val_points_domain', 'val_values_domain', 'val_source_terms_domain', 'coefs'])


In [200]:
def flatten_coefs_dict(coefs_dict):
    """Flatten geometry coefficient dict safely and always return valid tensor."""
    out_list = []
    for k, v in coefs_dict.items():
        if isinstance(v, np.ndarray):
            out_list.extend(v.reshape(-1).astype(float).tolist())
        elif np.isscalar(v):
            out_list.append(float(v))
        else:
            # Ignore unknown items but DO NOT break shape consistency
            continue

    if len(out_list) == 0:
        # fallback: one dummy coef to preserve tensor shape
        out_list = [0.0]

    return torch.tensor(out_list, dtype=torch.float32)


class NonlinearPoissonDataset(Dataset):
    def __init__(self, data_list, split="train", n_points=None):
        """
        n_points: None → use all points
                  int  → randomly sample that many points per item
        """
        self.data_list = data_list
        self.split = split
        self.n_points = n_points

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        item = self.data_list[idx]

        # ----------------------------------------------------------
        # Load raw arrays
        # ----------------------------------------------------------
        coords = item[f"{self.split}_points_domain"]        # (P,2)
        f      = item[f"{self.split}_source_terms_domain"]  # (P,)
        d      = item[f"{self.split}_distances_domain"]     # (P,)
        u      = item[f"{self.split}_values_domain"]        # (P,)

        # ----------------------------------------------------------
        # 🔥 Optional point subsampling
        # ----------------------------------------------------------
        P = coords.shape[0]
        if (self.n_points is not None) and (self.n_points < P):
            idxs = np.random.choice(P, self.n_points, replace=False)
            coords = coords[idxs]
            f      = f[idxs]
            d      = d[idxs]
            u      = u[idxs]

        # ----------------------------------------------------------
        # Convert to torch tensors (always ensure correct dimensionality)
        # ----------------------------------------------------------
        coords = torch.as_tensor(coords, dtype=torch.float32)

        f = torch.as_tensor(f, dtype=torch.float32).reshape(-1, 1)
        d = torch.as_tensor(d, dtype=torch.float32).reshape(-1, 1)
        u = torch.as_tensor(u, dtype=torch.float32).reshape(-1, 1)

        # ----------------------------------------------------------
        # Geometry coefficients (always produce consistent C)
        # ----------------------------------------------------------
        coefs = flatten_coefs_dict(item["coefs"])           # (C,)
        coefs = coefs.unsqueeze(0).expand(coords.shape[0], -1)

        # ----------------------------------------------------------
        # Stack features → (P, Ctotal)   where Ctotal = 1 + 1 + C
        # ----------------------------------------------------------
        try:
            features = torch.cat([f, d, coefs], dim=-1)
        except Exception:
            print("DEBUG shapes:")
            print("coords:", coords.shape)
            print("f:", f.shape)
            print("d:", d.shape)
            print("coefs:", coefs.shape)
            raise

        # Output: no batch dimension → DataLoader adds it automatically
        return coords, features, u


In [163]:
print(type(sample["coefs"]))
print(sample["coefs"])


<class 'dict'>
{'seed': 1, 'c1': np.float32(-0.0007273823), 'c2': np.float32(-0.0042264136), 'r0': 1.0, 'beta': array([-1.1116347 ,  0.66393006], dtype=float32), 'mu_1': array([-0.84975487, -1.102674  ], dtype=float32), 'mu_2': array([ 0.9603709, -1.4707267], dtype=float32), 'b': array([ 0.5957165 ,  0.4740579 , -0.05061221, -0.35185003,  0.6810765 ],
      dtype=float32)}


In [162]:
coords0, feats0, u0 = next(iter(train_loader))
print(coords0.shape, feats0.shape, u0.shape)

torch.Size([4, 2048, 2]) torch.Size([4, 2048, 17]) torch.Size([4, 2048, 1])


In [201]:
# Simple 90/10 split
n_total = len(raw_data)
n_train = int(0.9 * n_total)
train_raw = raw_data[:n_train]
val_raw   = raw_data[n_train:]

# 🔥 raw_data does NOT have "val_points_domain" keys → must use split="train"
train_ds = NonlinearPoissonDataset(train_raw, split="train", n_points=256)
val_ds   = NonlinearPoissonDataset(val_raw,   split="train", n_points=256)

def collate_fn(batch):
    """
    Explicit collate for safety.
    """
    coords, feats, u = zip(*batch)

    coords = torch.stack(coords, dim=0)  # (B, P, 2)
    feats  = torch.stack(feats,  dim=0)  # (B, P, C)
    u      = torch.stack(u,      dim=0)  # (B, P, 1)

    return coords, feats, u

train_loader = DataLoader(
    train_ds, batch_size=4, shuffle=True,
    num_workers=2, collate_fn=collate_fn
)

val_loader = DataLoader(
    val_ds, batch_size=4, shuffle=False,
    num_workers=2, collate_fn=collate_fn
)

coords0, feats0, u0 = next(iter(train_loader))
print("coords batch:", coords0.shape)
print("features batch:", feats0.shape)
print("u batch:", u0.shape)


coords batch: torch.Size([4, 256, 2])
features batch: torch.Size([4, 256, 17])
u batch: torch.Size([4, 256, 1])


# MODEL

In [202]:



class GNOPoissonModel(nn.Module):
    def __init__(
        self,
        in_channels=17,      # number of input features per point
        hidden_channels=64,  # internal GNO width
        out_channels=1,      # final scalar u(x)
        n_layers=4,
        radius=0.2,
        coord_dim=2,
    ):
        super().__init__()

        # 1) Project raw features (17) -> hidden_channels
        self.input_proj = nn.Linear(in_channels, hidden_channels)

        # 2) Pure GNO stack, all with same in/out = hidden_channels
        gno_layers = []
        for _ in range(n_layers):
            gno_layers.append(
                GNOBlock(
                    in_channels=hidden_channels,
                    out_channels=hidden_channels,
                    coord_dim=coord_dim,
                    radius=radius,
                    use_open3d_neighbor_search=False,
                )
            )
        self.gno_layers = nn.ModuleList(gno_layers)

        # 3) Pointwise head hidden_channels -> 1
        self.head = nn.Sequential(
            nn.Linear(hidden_channels, 32),
            nn.ReLU(),
            nn.Linear(32, out_channels),
        )

    # ------------------------------------------------------------------
    #            🔥 Completely custom, batch-safe forward
    # ------------------------------------------------------------------
    def forward(self, coords, feats):
        """
        coords : (B, N, 2) or (N, 2)
        feats  : (B, N, C_in) or (N, C_in)
        Returns:
            u_pred : (B, N, 1)
        """

        # Ensure batch dimension
        if coords.ndim == 2:   # (N, 2)
            coords = coords.unsqueeze(0)
        if feats.ndim == 2:    # (N, C)
            feats = feats.unsqueeze(0)

        B, N, _ = coords.shape
        _, Nf, Cin = feats.shape
        assert N == Nf, f"coords and feats mismatch: {coords.shape} vs {feats.shape}"

        outputs = []

        # IMPORTANT:
        # GNOBlock / neighbor_search expects 2D tensors (N, d)
        # so we run per-sample and stack.
        for b in range(B):
            y = coords[b]        # (N, 2)
            x = coords[b]        # (N, 2)
            h = feats[b]         # (N, Cin)

            # (a) project to hidden dimension
            h = self.input_proj(h)    # (N, hidden_channels)

            # (b) pass through all GNO blocks, all in/out = hidden_channels
            for layer in self.gno_layers:
                h = layer(y, x, h)    # all shapes (N, hidden_channels)

            # (c) pointwise head to scalar output
            h = self.head(h)         # (N, 1)

            outputs.append(h.unsqueeze(0))  # (1, N, 1)

        # stack back into (B, N, 1)
        return torch.cat(outputs, dim=0)


In [203]:
# Build model
in_channels = feats0.shape[-1]
print("Input feature channels:", in_channels)

model = GNOPoissonModel(
    in_channels=in_channels,
    hidden_channels=17,
    coord_dim=2,
    radius=0.2,
    n_layers=3,
).to(device)

print(model)



Input feature channels: 17
GNOPoissonModel(
  (input_proj): Linear(in_features=17, out_features=17, bias=True)
  (gno_layers): ModuleList(
    (0-2): 3 x GNOBlock(
      (pos_embedding): SinusoidalEmbedding()
      (neighbor_search): NeighborSearch()
      (integral_transform): IntegralTransform(
        (channel_mlp): LinearChannelMLP(
          (fcs): ModuleList(
            (0): Linear(in_features=256, out_features=128, bias=True)
            (1): Linear(in_features=128, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=128, bias=True)
            (3): Linear(in_features=128, out_features=17, bias=True)
          )
        )
      )
    )
  )
  (head): Sequential(
    (0): Linear(in_features=17, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [204]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = nn.MSELoss()


In [205]:
coords0, feats0, u0 = next(iter(train_loader))

print("coords0", type(coords0), coords0.shape)
print("feats0", type(feats0))
print("u0", type(u0))


coords0 <class 'torch.Tensor'> torch.Size([4, 256, 2])
feats0 <class 'torch.Tensor'>
u0 <class 'torch.Tensor'>


In [206]:
print("GNOBlock.forward signature:\n")
print(inspect.signature(GNOBlock.forward))

GNOBlock.forward signature:

(self, y, x, f_y=None)


In [207]:
print(inspect.getsource(GNOBlock.forward))


    def forward(self, y, x, f_y=None):
        """Compute a GNO neighbor search and kernel integral transform.

        Parameters
        ----------
        y : torch.Tensor of shape [n, d1]
            n points of dimension d1 specifying
            the space to integrate over.
            If batched, these must remain constant
            over the whole batch so no batch dim is needed.
        x : torch.Tensor of shape [m, d1], default None
            m points of dimension d1 over which the
            output function is defined. Must share domain
            with y
        f_y : torch.Tensor of shape [batch, n, d2] or [n, d2], default None
            Function to integrate the kernel against defined
            on the points y. The kernel is assumed diagonal
            hence its output shape must be d3 for the transforms
            (b) or (d). If None, (a) is computed.

        Output
        ----------
        out_features : torch.Tensor of shape [batch, m, d3] or [m, d3]
     

In [208]:
print(inspect.signature(GNOBlock.forward))
print(inspect.getsource(GNOBlock.forward))


(self, y, x, f_y=None)
    def forward(self, y, x, f_y=None):
        """Compute a GNO neighbor search and kernel integral transform.

        Parameters
        ----------
        y : torch.Tensor of shape [n, d1]
            n points of dimension d1 specifying
            the space to integrate over.
            If batched, these must remain constant
            over the whole batch so no batch dim is needed.
        x : torch.Tensor of shape [m, d1], default None
            m points of dimension d1 over which the
            output function is defined. Must share domain
            with y
        f_y : torch.Tensor of shape [batch, n, d2] or [n, d2], default None
            Function to integrate the kernel against defined
            on the points y. The kernel is assumed diagonal
            hence its output shape must be d3 for the transforms
            (b) or (d). If None, (a) is computed.

        Output
        ----------
        out_features : torch.Tensor of shape [batch, 

In [ ]:
from tqdm import tqdm

def run_epoch(loader, train=True):
    model.train() if train else model.eval()

    total_loss = 0.0
    n_batches = 0

    pbar = tqdm(loader, desc="Train" if train else "Val", leave=False)

    for coords, feats, u in pbar:

        # -------- Device + dtype --------
        coords = coords.to(device).float()   # (B, N, 2)
        feats  = feats.to(device).float()    # (B, N, C)
        u      = u.to(device).float()        # (B, N, 1)

        if train:
            optimizer.zero_grad(set_to_none=True)

        with torch.set_grad_enabled(train):
            # -------- Forward pass --------
            u_pred = model(coords, feats)

            loss = criterion(u_pred, u)

            # -------- Backward pass --------
            if train:
                loss.backward()

                # Gradient explosion protection
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                optimizer.step()

        # -------- Accumulate loss --------
        total_loss += loss.item()
        n_batches += 1

        pbar.set_postfix({"loss": f"{total_loss / n_batches:.3e}"})

    return total_loss / max(1, n_batches)


# ----------------------------------------------------------------------------
# OUTER TRAINING LOOP — FIXED
# ----------------------------------------------------------------------------

n_epochs = 20
train_losses = []
val_losses = []

epoch_bar = tqdm(range(1, n_epochs + 1), desc="Epochs")

for epoch in epoch_bar:

    train_loss = run_epoch(train_loader, train=True)
    val_loss   = run_epoch(val_loader,   train=False)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    epoch_bar.set_postfix({
        "train": f"{train_loss:.3e}",
        "val":   f"{val_loss:.3e}",
    })

    print(f"[Epoch {epoch:03d}] "
          f"train MSE: {train_loss:.4e}   val MSE: {val_loss:.4e}")


Train:   0%|          | 0/2250 [00:00<?, ?it/s]


Train:   0%|          | 1/2250 [00:02<1:19:15,  2.11s/it, loss=2.251e+02]


Train:   0%|          | 2/2250 [00:03<58:25,  1.56s/it, loss=8.745e+03]  


Train:   0%|          | 3/2250 [00:04<51:39,  1.38s/it, loss=6.327e+03]


Train:   0%|          | 4/2250 [00:05<48:30,  1.30s/it, loss=4.800e+03]

...


Train:  71%|███████   | 1594/2250 [31:29<12:59,  1.19s/it, loss=1.674e+01]


Train:  71%|███████   | 1595/2250 [31:31<12:56,  1.19s/it, loss=1.673e+01]


Train:  71%|███████   | 1596/2250 [31:32<12:54,  1.18s/it, loss=1.672e+01]


Train:  71%|███████   | 1597/2250 [31:33<12:53,  1.18s/it, loss=1.671e+01]


Train:  71%|███████   | 1598/2250 [31:34<12:57,  1.19s/it, loss=1.670e+01]

In [ ]:
coords0, feats0, u0 = next(iter(train_loader))
print(coords0.shape)
print(feats0.shape)
print(u0.shape)

torch.Size([4, 2048, 2])
torch.Size([4, 2048, 17])
torch.Size([4, 2048, 1])
